In [5]:
from utils.utils import *
from utils.utils_nn import *
from utils.utils_baseline_svm import *
from utils.Rnn_model import Rnn_model

import numpy as np
np.random.seed(seed())
import pandas as pd

import tensorflow as tf
from tensorflow.contrib import rnn

import os

# import random
# random.seed(seed())

# import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn import svm

from collections import Counter, OrderedDict

In [17]:
# setup hyper paramteres and classifiers configurations
# SVM ========================
kwargs_lin_clf = nice_dict({'mk_chars': True, 
                            'model': 'linear', 
                            'char_filter': 100, 'allowed_chars': None, 
                            'mk_ngrams': True, 'ngram_width': 5, 
                            'ngram_filter': 10, 'allowed_ngrams': None, 
                            'keep_infreq_labels': False, 'label_count_thresh': 10, 
                            'valid_ratio': 0.25, 
                            'scale_func': unscale, 'to_permute': True, })

kwargs_svm = nice_dict({'C': 0.1,  # penalty term
                        'decision_function_shape': 'ovr',  # one-vs-rest (‘ovr’) / one-vs-one (‘ovo’) 
                        'random_state': seed(), 
                        'kernel': 'linear', 
                        'gamma': 'auto' ,  # kernel coef for ‘rbf’, ‘poly’ and ‘sigmoid’. ‘auto’ -> 1/n_features
                        'probability': True,  # enable probability estimates 
                        'shrinking': True,  # use the shrinking heuristic 
                        'max_iter': -1  # -1 mean no limitation 
                        })

# RNN ========================
kwargs_neural_data_init = \
    {'mk_chars': True, 
               'model': 'neural', 
               'char_filter': 100, 'allowed_chars': None, 
               'mk_ngrams': False, 'ngram_width': 5, 
               'ngram_filter': 10, 'allowed_ngrams': None, 
               'keep_infreq_labels': False, 'label_count_thresh': 10, 
               'valid_ratio': 0.25, 
               'scale_func': unscale, 'to_permute': True, }

kwargs_simple_rnn = nice_dict({
    # log
    'log_dir': 'experiments/logdir_exper_4_20_GRU/', 
    'del_log': False, 
    # preprocessing and data
    'top_k': 5, 
    'seed': seed(), 
    # learning hyper-params
    'learn_rate': 1E-2, 
    'dynamic_learn_rate': False, 
    'rnn_type': 'GRU',
    'bidirection': False, 
    'char_embed_dim': 4, 
    'one_hot': True,
    'hidden_state_size': 128, 
    'keep_prob': 0.7, 
    # noisy activation hyper params
    'activation_function': 'noisy_tanh',  # tf.tanh / 'noisy_tanh'
    'learn_p_delta_scale': False,   # noise scale param in noisy activation
    'noise_act_alpha': 0.9,  # mixing in the linear activation
    'noise_act_half_normal': False,
    # regularization constants
    'l2_weight_reg': 1E-3, 
    'target_rep': True, 
    'target_rep_weight': 0.3, 
    # training settings
    'epochs': 20,
    'summary_step': 10, 
    'save_step': np.inf,
    'to_save': False, 
    'verbose_summary': False
})

# GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=0.9,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_wieght_reg=1.0E-03,target_rep_weight=0.3
rnn_dir_name = 'GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=0.9,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_wieght_reg=1.0E-03,target_rep_weight=0.3/'

In [3]:
# SVM ========================
# vectorizer transforms dict into sparse matrix
v = DictVectorizer(sparse=True)

# load data
x_train_svm, x_val_svm, \
    y_train_svm, y_val_svm, \
    allowed_ngrams = \
    data_load_preprocess(**{**kwargs_lin_clf, 
                            **{'linear_counters': False}})

# create and fit classifier
# create a sparse X matrix with character and n-grams features
X_train_svm = v.fit_transform([Counter(elem) for elem in x_train_svm])
X_val_svm = v.transform([Counter(elem) for elem in x_val_svm])

print('X_train_svm (sparse) matrix, of size {} by {} has been created.'
      .format(X_train_svm.get_shape()[0], 
              X_train_svm.get_shape()[1]))  # vectorized

svm_clf = svm.SVC(**kwargs_svm)

# print(svm_clf)

svm_clf.fit(X_train_svm, y_train_svm)

The are 2919 observations
Sampling from allowed 82 labels
82 labels in the validation set, with
1587 potential observation to draw from.
365 observations sampled for validation
1222 observations for training
The ratio of validation to *training* is about 0.299
X_train_svm (sparse) matrix, of size 1222 by 1886 has been created.
SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=2178, shrinking=True,
  tol=0.001, verbose=False)


SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=2178, shrinking=True,
  tol=0.001, verbose=False)

In [ ]:
# array to hold log probabilities (takes a bit longer to calc)
pred_prob = svm_clf.predict_log_proba(X_val_svm)
# makes y into array with the same shape as the log prob
y_val_svm_dense = y_to_dense(y=y_val_svm, 
                         classes=svm_clf.classes_)

print('Accuracy on validation set is {:.3f}'.
      format(svm_clf.score(X_val_svm, 
                           y_val_svm)))

print('Mean Reciprocal Rank is {:.3f}'
      .format(mean_reciprocal_rank(y_val_svm_dense, 
                                   pred_prob)))

In [ ]:
mrr = mean_reciprocal_rank(
    y_val_svm_dense, 
    pred_prob)

acc1 = in_top_k(
    y_val_svm_dense, 
    pred_prob, 
    1)

top5 = in_top_k(
    y_val_svm_dense, 
    pred_prob, 
    5)

print(mrr, top5, acc1, acc)

In [ ]:
keep_first_k_chars(input=x_train_svm, k=7, 
                   model='linear', 
                   ngram_width=kwargs_lin_clf.ngram_width, 
                   mk_ngrams=kwargs_lin_clf.mk_ngrams, 
                   allowed_ngrams=allowed_ngrams)

In [4]:
# RNN
# load data (only validation data, since we're not training RNN now)
_, _, x_feed_val_rnn, y_feed_val_rnn,\
    char_int, char_int_inv, label_int, label_int_inv, \
    statistics_dict =\
    data_load_preprocess(**kwargs_neural_data_init)

kwargs_simple_rnn = {**kwargs_simple_rnn, 
                      **statistics_dict}

kwargs_simple_rnn = nice_dict({**kwargs_simple_rnn, 
                               **{'scale_func': kwargs_neural_data_init['scale_func'], 
                                  'keep_infreq_labels': kwargs_neural_data_init['keep_infreq_labels']}})

# validation data
X_val_rnn, _, Y_val_rnn = index_transorm_xy(x=x_feed_val_rnn, 
                                    y=y_feed_val_rnn, 
                                    char_int=char_int, 
                                    label_int=label_int, 
                                    **kwargs_simple_rnn)

kwargs_feed_dict_test = {'x': X_val_rnn, 
                         'y': Y_val_rnn}

# create classifier and load weights
Rnn_clf = Rnn_model(
    feed_dict_test=kwargs_feed_dict_test, 
    **{**kwargs_simple_rnn, 
       **{}}  # 'epochs': 100
)


# Rnn_clf.close_session()

The are 2919 observations
Sampling from allowed 82 labels
82 labels in the validation set, with
1587 potential observation to draw from.
365 observations sampled for validation
1222 observations for training
The ratio of validation to *training* is about 0.299


In [8]:
hparam_str = make_hparam_string(**kwargs_simple_rnn)

In [18]:
print(hparam_str)
print(rnn_dir_name)
print(hparam_str == rnn_dir_name)

GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=0.9,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_weight_reg=1.0E-03,target_rep_weight=0.3/
GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=0.9,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_wieght_reg=1.0E-03,target_rep_weight=0.3/
False


In [20]:
# Rnn_clf.restore(cp_path=os.path.join(kwargs_simple_rnn.log_dir, hparam_str))
Rnn_clf.restore(cp_path=os.path.join(kwargs_simple_rnn.log_dir, rnn_dir_name))
# Rnn_clf.restore(cp_path=rnn_dir_name)

Loading variables from experiments/logdir_exper_4_20_GRU/GRU,bidir=F,noisy_tanh,learn_p=F,noise_alpha=0.9,noise_half_normal=F,keep_infreq_labels=F,learn_rate=1.0E-02,keep_prob=0.7,one_hot,hidden_state_size=128,l2_wieght_reg=1.0E-03,target_rep_weight=0.3/


Exception: no checkpoint found

In [ ]:
[accuracy, cost, recip_rank, top_k] = rnn_new.run_eval()
print('accuracy is {:.5f}'.format(accuracy))

In [ ]:
rnn_new.update_test_dict(kwargs_feed_dict_train)
[accuracy, cost, recip_rank, top_k] = rnn_new.run_eval()
print('accuracy is {:.5f}'.format(accuracy))

In [ ]:
keep_first_k_chars(input=X_train, k=2, 
                       model='neural', 
                       char_int=char_int)

In [ ]:
# generate first_k_characters sequence

In [ ]:
# collect evaulation metrics for sequence length

In [ ]:
# create Pandas DataFrame